In [2]:
# Importing the libraries 
import pandas as pd
import numpy as np
import random
import pickle
import re
import regex  # for better, more capbale regex api
import os
import zipfile
import more_itertools
from itertools import chain
import datetime
import time
from statsmodels.stats.proportion import proportion_confint
# active labeler related
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import ComplementNB  # corrects for class imbalance, SGD is pretty good too
from sklearn.pipeline import Pipeline
from superintendent import ClassLabeller
from IPython.display import display, Markdown

pd.set_option('display.max_colwidth', None)  # so we can peak at data and spot verify

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1,2))),
    
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])
print('done')

done


In [3]:
# Set up columns to keep, fields, locations for writing
rootpath = "/hdd/work/d4ad_standardization/"
processedpath = "D4AD_Standardization/data/processed/"
externalpath = "D4AD_Standardization/data/external/"
interimpath = "D4AD_Standardization/data/interim/"

content_is = "standardized_descriptions_and_degree_funding_type"

print('done')

done


In [14]:

filepath = content_is + ".csv" # builds off of notebook 6.0 work

SKIP_THIS = True # helps me be able to run all and not worry about pulling things
# I already know I have on disk

df = pd.read_csv(rootpath + interimpath + filepath)
print('done2')

done2


In [15]:
pd.set_option('display.max_rows', False)

the_df = df #df.sample(n=10000, random_state=42)

In [16]:
# 2) We need to create a length of job search assistance field; WIOA, at minimum, offers
# 6 months, but some counties or states require more; this possibly migth be mentioned
# and we would like to extract that.

# We first repeat the regex machinery from notebook 6.0 so taht we can
# more easily search and find rows with training realted field data.


# TODO: try out basic .str.match from Pandas first, this machinery might not
# be needed.
def make_term_grouped_regex(term="", right_regex="", left_regex=""):
    mystr = left_regex + '(' +\
                re.escape(term) +\
            ')' + right_regex
    return mystr

def make_grouped_regexes(replacement, left_regex="", right_regex=""):
    return (make_term_grouped_regex(left_regex=left_regex,
                                    term=key,
                                    right_regex=right_regex)\
            for key in replacement.keys()
    )

def construct_map(label_mapper=label_mapper):
    return {
        **dict(zip(label_mapper.abbreviation, label_mapper.expanded))
    }

abbrevation_pattern =\
    regex.compile(
        "(?p)" +
        "|".join(   # match words at start of string
            make_grouped_regexes(replacement_map, left_regex=r'^', right_regex=r'[\s:]')
        ) + "|" +\
        "|".join(   # match words surrounded by spaces
            make_grouped_regexes(replacement_map, left_regex=r'\s', right_regex=r'\s')
        ) + "|" +\
        "|".join(   # match words that make up entire fields, e.g. 'Nursing'
            make_grouped_regexes(replacement_map, left_regex=r'^', right_regex=r'$')
        ) + "|" +\
        "|".join(   # match words at end of string preceded by space or slash
            make_grouped_regexes(replacement_map, left_regex=r'[\s/]', right_regex=r'$')
        ) + "|" +\
        "|".join(   # match words within string that follow a slash, end with a space or slash
            make_grouped_regexes(replacement_map, left_regex=r'/', right_regex=r'[\s/]')
        )
    )

def multiple_mapper(string):
    return abbrevation_pattern.sub(
        lambda x: \
        x.group().replace( # replace the found string
            more_itertools.first_true(x.groups() # where the first matched group...
        ),  replacement_map[more_itertools.first_true(x.groups())] # ... is replaced with the lookup
    ), string)
print('done1')

done1


In [1]:
training_regex =\
    """
    (job[\s\b.].*?search                # job search, mentioned in an order
     |assist[\w\s\b\.].*?employ         # or assitance gaining employment
     |employ[\w\s\b\.].*?assist)        # or employment assistance :)
    """

get_surrounding_prior_words_regex =\
    """
    (?P<prior>\w+\W+){0,4}\b
    """

get_surrounding_after_words_regex =\
    """
    \b(?P<after>\W+\w+){0,4})
    """

the_regex = get_surrounding_prior_words_regex +\
            training_regex                    +\
            get_surrounding_after_words_regex

the_df['minimum_six_months'] =\
    False

the_df[wioa_indices, 'minimum_six_months'] =\
    True

the_df['additional_search_mention'] =\
    None
    
# TODO: finish out this logic, should create a prior, after column
# that contain time/date information that can be examined (e.g. 
# contains four, and month, so four months, etc.)
wioa_indices = the_df['IS_WIOA'] == True
mention_training_indices = the_df.loc[wioa_indices, 'DESCRIPTION']\
                                 .str\
                                 .extractall(the_regex, flags=re.I|re.VERBOSE)

# Of those mentioning job searches, do they mention how long?
how_long_regex =\
    """
     .*\d\d.*
    |one
    |three
    |four
    |five
    |six
    |seven
    |eight
    |nine
    |ten
    |eleven
    |twelve
    |thirteen
    |fourteen
    |fifteen
    |sixteen
    """

# I want
#     0 a number
#     0 followed by a length of time
#     0  followed or preceded by a mention of the training regex

the_df.loc[wioa_indices & mention_training_indices, :]


NameError: name 'the_df' is not defined

In [18]:
# 4)
# Now we write out the verfiied results
# ... finally we can write this out as our first complete lookup table
# for the NAME field
write_out = draft_output

print(
    "We're writing ...",
    write_out.columns
)

content_is = "with_job_search_durations"

# shuffe the rows to better remove temporal baises
write_out =\
    write_out.sample(frac=1, random_state=42, axis=0).reset_index(drop=True)

write_out.to_csv(rootpath + interimpath + content_is + ".csv",
                index = False,
                chunksize = 10000)

write_out.to_excel(rootpath + processedpath + content_is + ".xls",
            sheet_name="Standardized Descriptions",
            index=False)
print('done')

We're writing ... Index(['IS_WOIA', 'Mentioned_Certificate', 'Mentioned_Associates',
       'STANDARDIZED_DESCRIPTION', 'STANDARDIZED_FEATURESDESCRIPTION',
       'CLEANED_STANDARDIZED_NAME_1', 'STANDARDIZEDNAME', 'STANDARDIZEDNAME_1',
       'DESCRIPTION', 'FEATURESDESCRIPTION', 'NAME_1', 'NAME'],
      dtype='object')
done
